In [ ]:
from Preprocess import *
from Model import *
from DataAnalysis import *

In [ ]:
dic_csv = read_data()
df_var = make_dataframe(dic_csv)

In [ ]:
concat_train_test(dic_csv, df_var)
loans = Merge(df_var, haveCategorical=True)

In [ ]:
original_loan_per_year = loan_per_year(df_var['loan'], isMerged = False)

In [ ]:
histDf(original_loan_per_year)

In [ ]:
getCorr(original_loan_per_year[1993])

In [ ]:
model_year(original_loan_per_year[1993], original_loan_per_year[1994])


In [ ]:
loans

In [ ]:
X1